<a href="https://colab.research.google.com/github/Subina00/blockchain-project/blob/main/Copy_of_Assessment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
## Import statements
import hashlib
import time

class Block:
  # Block instance attributes
    def __init__(self, index, previous_hash, timestamp, data, proof):
        self.index = index
        self.previous_hash = previous_hash
        self.timestamp = timestamp
        self.data = data
        self.proof = proof
        self.hash = self.calculate_hash()

    def calculate_hash(self):

        #Combine block attributes into a string and return the SHA-256 hash.

        block_content = f"{self.index}{self.previous_hash}{self.timestamp}{self.data}{self.proof}"
        return hashlib.sha256(block_content.encode()).hexdigest()



class Blockchain:
    def __init__(self):
        self.chain = [self.create_genesis_block()]
        self.difficulty = 4  # Number of leading zeros required in the hash

    def create_genesis_block(self):
        """
        Creates the first block with static initial values.
        """
        return Block(0, "0", time.time(), "Genesis Block", 0)

    def get_latest_block(self):
        return self.chain[-1]

    def add_block(self, new_block):
        # TODO: Add a new block to the chain
        # Hint: Set the new block's previous_hash to the hash of the latest block
        pass

    def proof_of_work(self, block):
        """
        Proof-of-work: increment proof until hash starts with N zeros.
        """
        block.proof = 0
        block.hash = block.calculate_hash()
        while not block.hash.startswith('0' * self.difficulty):
            block. proof += 1
            block.hash = block.calculate_hash()
        return block

    def add_data(self, data):
        # TODO: Create a new block with the provided data, perform proof of work, and add it to the chain
        pass

    def is_chain_valid(self):
        # TODO: Validate the integrity of the blockchain
        # Hint: Check that each block's hash is correct and that the previous_hash matches the hash of the previous block
        pass


# Example Usage
if __name__ == "__main__":
    blockchain = Blockchain()

    print("Mining block 1...")
    blockchain.add_data("Transaction data for Block 1")

    print("Mining block 2...")
    blockchain.add_data("Transaction data for Block 2")

    print("\nBlockchain validity:", blockchain.is_chain_valid())

    for block in blockchain.chain:
        print(f"Block {block.index} | Hash: {block.hash} | Previous Hash: {block.previous_hash}")

Mining block 1...
Mining block 2...

Blockchain validity: None
Block 0 | Hash: 3f08822353634a775a2f00dd201b6afffaaedb07307c1782518beb6a7f82cb54 | Previous Hash: 0
